## Première demande de Fernanda : Exécuter 4 requetes sql pour alimenter un dashboard pour l'entreprise.

In [2]:
import sqlite3 as sq
from functools import wraps
from typing import Any, Sequence, Optional
import pandas as pd

DATABASE_PATH = "../data/olist.db"

def db_connection(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Permet de passer un autre chemin si besoin: sql_df(query, params, db_path="...")
        db_path = kwargs.pop("db_path", DATABASE_PATH)
        with sq.connect(db_path) as conn:
            conn.execute("PRAGMA foreign_keys = ON;")
            return func(conn, *args, **kwargs)  # commit/rollback gérés par le context manager
    return wrapper

@db_connection
def sql_df(conn: sq.Connection, query: str, params: Optional[Sequence[Any]] = None) -> pd.DataFrame:
    """SELECT → DataFrame (prend en charge les paramètres SQL via '?')."""
    return pd.read_sql_query(query, conn, params=params)


### En excluant les commandes annulées, quelles sont les commandes récentes de moins de 3 mois que les clients ont reçues avec au moins 3 jours de retard ?

In [3]:
req1 = """
    WITH o AS (
        SELECT
            order_id,
            customer_id,
            order_status,
            order_purchase_timestamp,
            order_delivered_customer_date,
            order_estimated_delivery_date,
            -- retard en jours
            (julianday(order_delivered_customer_date) - julianday(order_estimated_delivery_date)) AS delay_days
        FROM orders
        WHERE order_delivered_customer_date IS NOT NULL
            AND order_estimated_delivery_date IS NOT NULL
    )
    SELECT
        order_id,
        customer_id,
        order_purchase_timestamp,
        order_delivered_customer_date,
        order_estimated_delivery_date,
        ROUND(delay_days, 2) AS delay_days
    FROM o
    WHERE order_status <> 'canceled'
        AND delay_days >= 3
        AND DATE(order_purchase_timestamp) >= DATE('now','-3 months')
    ORDER BY order_purchase_timestamp DESC;
"""
sql_df(req1)

,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delay_days


### Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100 000 Real sur des commandes livrées via Olist ?

In [4]:
req2 = """
    WITH delivered AS (
            SELECT order_id
            FROM orders
            WHERE order_status = 'delivered'
        ),
        seller_rev AS (
            SELECT
                oi.seller_id,
                COUNT(DISTINCT oi.order_id) AS n_orders,
                SUM(oi.price) AS revenue_products,
                SUM(oi.price + oi.freight_value) AS revenue_products_plus_freight
            FROM order_items oi
            JOIN delivered d USING (order_id)
            GROUP BY oi.seller_id
        )
        SELECT
            s.seller_id,
            n_orders,
            ROUND(revenue_products, 2) AS revenue_products_brl,
            ROUND(revenue_products_plus_freight, 2) AS revenue_products_plus_freight_brl
        FROM seller_rev sr
        JOIN sellers s ON s.seller_id = sr.seller_id
        WHERE revenue_products > 100000
        ORDER BY revenue_products DESC;
"""
sql_df(req2)

,seller_id,n_orders,revenue_products_brl,revenue_products_plus_freight_brl
0,4869f7a5dfa277a7dca6462dcf3b52b2,1124,226987.93,247007.06
1,53243585a1d6dc2643021fd1853d8905,348,217940.44,230797.02
2,4a3ca9315b744ce9f8e9374361493884,1772,196882.12,231220.43
3,fa1c13f2614d7b5c4749cbc52fecda94,578,190917.14,200833.50
4,7c67e1448b00f6e969d365cea6b010ab,973,186570.05,237806.69
5,7e93a43ef30c4f03f38b393420bc753a,319,165981.49,171973.55
6,da8622b14eb17ae2831f4ac5b9dab84a,1311,159816.87,184706.78
7,7a67c85e85bb2ce8582c35f2203ad736,1145,139658.69,160278.52
8,1025f0e2d44d7041d6cf58b6550e0bfa,910,138208.56,171924.96
9,955fee9216a65b617aa5c0531780ce60,1261,131836.71,156606.48


### Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui sont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30 produits) ?

In [6]:
req3 = """
    WITH ref AS (
        SELECT DATE(MAX(order_purchase_timestamp)) AS ref_date
        FROM orders
    ),
    delivered AS (
        SELECT o.order_id, o.order_purchase_timestamp
        FROM orders o
        WHERE o.order_status = 'delivered'
    ),
    seller_sales AS (
        SELECT
            oi.seller_id,
            MIN(d.order_purchase_timestamp) AS first_sale_ts,
            COUNT(oi.order_item_id) AS units_sold
        FROM order_items oi
        JOIN delivered d ON d.order_id = oi.order_id
        GROUP BY oi.seller_id
    )
    SELECT
        ss.seller_id,
        DATE(ss.first_sale_ts) AS first_sale_date,
        ss.units_sold,
        CAST(julianday((SELECT ref_date FROM ref)) - julianday(ss.first_sale_ts) AS INT) AS days_since_first_sale
    FROM seller_sales ss
    WHERE DATE(ss.first_sale_ts) >= DATE((SELECT ref_date FROM ref), '-3 months')
        AND ss.units_sold > 30
    ORDER BY ss.units_sold DESC;
"""
sql_df(req3)

,seller_id,first_sale_date,units_sold,days_since_first_sale
0,d13e50eaa47b4cbe9eb81465865d8cfc,2018-08-04,68,73
1,81f89e42267213cb94da7ddc301651da,2018-08-08,52,69
2,240b9776d844d37535668549a396af32,2018-07-17,35,91


### Question : Quels sont les 5 codes postaux, enregistrant plus de 30 reviews, avec le pire review score moyen sur les 12 derniers mois ?

In [7]:
req4 = """
    WITH ref AS (
        SELECT DATE(MAX(review_creation_date)) AS ref_date
        FROM order_reviews
    ),
    recent_reviews AS (
        SELECT
            r.review_id,
            r.review_score,
            r.review_creation_date,
            o.customer_id
        FROM order_reviews r
        JOIN orders o ON o.order_id = r.order_id
        WHERE DATE(r.review_creation_date) >= DATE((SELECT ref_date FROM ref), '-12 months')
    ),
    reviews_with_zip AS (
        SELECT
            rr.review_id,
            rr.review_score,
            c.customer_zip_code_prefix
        FROM recent_reviews rr
        JOIN customers c ON c.customer_id = rr.customer_id
    )
    SELECT
        customer_zip_code_prefix,
        COUNT(*) AS review_count,
        ROUND(AVG(review_score), 2) AS avg_review_score
    FROM reviews_with_zip
    GROUP BY customer_zip_code_prefix
    HAVING COUNT(*) > 30
    ORDER BY avg_review_score ASC
    LIMIT 5;
"""
sql_df(req4)

,customer_zip_code_prefix,review_count,avg_review_score
0,22753,53,2.87
1,22723,31,3.00
2,28893,32,3.13
3,22770,38,3.18
4,13056,33,3.27
